In [429]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [419]:
dfd = pd.read_csv(r'demandeurs_3_revenus.csv', index_col = 0)

In [420]:
dfd=dfd.rename(columns={'Type de ressource\n': 'Type'})
dfd

,Numéro Unique,Ressources mensuelles\n,Type,Montant par type de ressource\n
0,001031606376411126,829.0,Allocation familiales,129.0
1,001031606376411126,829.0,Autres (hors APL ou AL),209.0
2,001031606376411126,829.0,Revenu de solidarité active (RSA),491.0
3,001031707754001033,759.0,Autres (hors APL ou AL),249.0
4,001031707754001033,759.0,Revenu de solidarité active (RSA),37.0
...,...,...,...,...
166753,9740116110885GDPUB,472.0,Revenu de solidarité active (RSA),472.0
166754,9740116110885GDPUB,472.0,Salaire ou revenu d'activité,0.0
166755,9740417140838GDPUB,1700.0,Salaire ou revenu d'activité,1700.0
166756,974081714984011487,764.0,Pension d'invalidité,764.0


In [421]:
dfd['Ressources mensuelles\n'].isna().sum()

3350

In [378]:
dfdbis=dfd.drop_duplicates(subset ="Numéro Unique")
dfdbis=dfdbis.dropna()
dfdbis

,Numéro Unique,Ressources mensuelles\n,Type,Montant par type de ressource\n
0,001011403466511146,2134.0,Allocation chômage / indemnités,240.0
4,0010117074877GDPUB,900.0,Allocation familiales,700.0
6,001021504965918214,944.0,Allocation familiales,129.0
8,0010217076033GDPUB,1200.0,Allocation chômage / indemnités,1200.0
9,001021707620301173,1200.0,Salaire ou revenu d'activité,1200.0
...,...,...,...,...
152671,9741117155529GDPUB,674.0,Revenu de solidarité active (RSA),674.0
152672,974111715873511487,1725.0,Retraite,1725.0
152673,9741216133916GDPUB,2000.0,Salaire ou revenu d'activité,2000.0
152674,9741216133920GDPUB,846.0,Retraite,846.0


### Calcul des ressources imposables mensuelles

In [379]:
values=["Retraite", "Allocation chômage / indemnités", "Salaire ou revenu d'activité", "Pension alimentaire reçue", "Autres (hors APL ou AL)", "Pension d\'invalidité"]
filtered_df = dfd[dfd.Type.isin(values)]
filtered_df

,Numéro Unique,Ressources mensuelles\n,Type,Montant par type de ressource\n
0,001011403466511146,2134.0,Allocation chômage / indemnités,240.0
2,001011403466511146,2134.0,Autres (hors APL ou AL),517.0
3,001011403466511146,2134.0,Salaire ou revenu d'activité,953.0
5,0010117074877GDPUB,900.0,Pension alimentaire reçue,200.0
8,0010217076033GDPUB,1200.0,Allocation chômage / indemnités,1200.0
...,...,...,...,...
152670,974091715322011487,2121.0,Salaire ou revenu d'activité,2083.0
152672,974111715873511487,1725.0,Retraite,1725.0
152673,9741216133916GDPUB,2000.0,Salaire ou revenu d'activité,2000.0
152674,9741216133920GDPUB,846.0,Retraite,846.0


In [380]:
shrank_values = filtered_df.groupby(by='Numéro Unique').sum()['Montant par type de ressource\n']

merged_df = filtered_df.merge(shrank_values, left_on='Numéro Unique', right_index=True)

merged_df = merged_df.rename(columns={'Montant par type de ressource\n_y': 'Ressources imposables mensuelles moyennes (RI)'})


In [381]:
merged_df

,Numéro Unique,Ressources mensuelles\n,Type,Montant par type de ressource\n_x,Ressources imposables mensuelles moyennes (RI)
0,001011403466511146,2134.0,Allocation chômage / indemnités,240.0,1710.0
2,001011403466511146,2134.0,Autres (hors APL ou AL),517.0,1710.0
3,001011403466511146,2134.0,Salaire ou revenu d'activité,953.0,1710.0
5,0010117074877GDPUB,900.0,Pension alimentaire reçue,200.0,200.0
8,0010217076033GDPUB,1200.0,Allocation chômage / indemnités,1200.0,1200.0
...,...,...,...,...,...
152670,974091715322011487,2121.0,Salaire ou revenu d'activité,2083.0,2083.0
152672,974111715873511487,1725.0,Retraite,1725.0,1725.0
152673,9741216133916GDPUB,2000.0,Salaire ou revenu d'activité,2000.0,2000.0
152674,9741216133920GDPUB,846.0,Retraite,846.0,846.0


In [382]:
merged_dfbis=merged_df.drop_duplicates(subset ="Numéro Unique")
merged_dfbis

,Numéro Unique,Ressources mensuelles\n,Type,Montant par type de ressource\n_x,Ressources imposables mensuelles moyennes (RI)
0,001011403466511146,2134.0,Allocation chômage / indemnités,240.0,1710.0
5,0010117074877GDPUB,900.0,Pension alimentaire reçue,200.0,200.0
8,0010217076033GDPUB,1200.0,Allocation chômage / indemnités,1200.0,1200.0
9,001021707620301173,1200.0,Salaire ou revenu d'activité,1200.0,1200.0
10,001041707857711126,800.0,Retraite,800.0,800.0
...,...,...,...,...,...
152670,974091715322011487,2121.0,Salaire ou revenu d'activité,2083.0,2083.0
152672,974111715873511487,1725.0,Retraite,1725.0,1725.0
152673,9741216133916GDPUB,2000.0,Salaire ou revenu d'activité,2000.0,2000.0
152674,9741216133920GDPUB,846.0,Retraite,846.0,846.0


In [383]:
bdd = dfdbis.merge(merged_dfbis, on='Numéro Unique', how='outer')
bdd=bdd[['Numéro Unique', 'Ressources mensuelles\n_x','Ressources imposables mensuelles moyennes (RI)']]
bdd=bdd.rename(columns={'Ressources mensuelles\n_x': 'RM Moyennes'})
bdd

,Numéro Unique,RM Moyennes,Ressources imposables mensuelles moyennes (RI)
0,001011403466511146,2134.0,1710.0
1,0010117074877GDPUB,900.0,200.0
2,001021504965918214,944.0,NaN
3,0010217076033GDPUB,1200.0,1200.0
4,001021707620301173,1200.0,1200.0
...,...,...,...
80874,9741117155529GDPUB,674.0,NaN
80875,974111715873511487,1725.0,1725.0
80876,9741216133916GDPUB,2000.0,2000.0
80877,9741216133920GDPUB,846.0,846.0


In [150]:
#Foyers dont les ressources mensuelles ne matchent pas le revenu imposable => Une part des ressources n'est pas imposable
merged_df[merged_df['Revenu imposable']!=merged_df['Ressources mensuelles']]

,Numéro Unique,Ressources mensuelles,Type,Montant par type de ressource_x,Revenu imposable
index,,,,,
29,002011403169610414,946.0,Salaire ou revenu d'activité,650.0,650.0
30,002011707708910411,1000.0,Retraite,1001.0,1001.0
32,0020117077711GDPUB,1850.0,Salaire ou revenu d'activité,1650.0,1650.0
34,002011707772510420,1982.0,Salaire ou revenu d'activité,1700.0,1700.0
37,002021707869210424,1371.0,Retraite,1671.0,1671.0
...,...,...,...,...,...
1285435,972121705572711572,1157.0,Retraite,1044.0,1044.0
1285437,9740317139610AL020,691.0,Pension d'invalidité,991.0,991.0
1285440,9740317140139AL020,3728.0,Pension alimentaire reçue,100.0,1828.0


In [151]:
#Foyers avec plusieurs type de revenus
merged_df[merged_df['Revenu imposable']!=merged_df['Montant par type de ressource_x']]


,Numéro Unique,Ressources mensuelles,Type,Montant par type de ressource_x,Revenu imposable
index,,,,,
3,001071606857001033,1570.0,Allocation chômage / indemnités,500.0,1570.0
4,001071606857001033,1570.0,Salaire ou revenu d'activité,1070.0,1570.0
7,001111708713716214,1868.0,Pension d'invalidité,723.0,1868.0
8,001111708713716214,1868.0,Retraite,1145.0,1868.0
13,0011217089230GDPUB,1800.0,Allocation chômage / indemnités,0.0,1800.0
...,...,...,...,...,...
1285430,972091705244411572,3002.0,Salaire ou revenu d'activité,1501.0,3002.0
1285440,9740317140139AL020,3728.0,Pension alimentaire reçue,100.0,1828.0
1285441,9740317140139AL020,3728.0,Salaire ou revenu d'activité,1728.0,1828.0


In [152]:
#Foyers avec un seul type de revenu
merged_df[~(merged_df['Revenu imposable']!=merged_df['Montant par type de ressource_x'])]


,Numéro Unique,Ressources mensuelles,Type,Montant par type de ressource_x,Revenu imposable
index,,,,,
0,0010216061927GDPUB,908.0,Retraite,908.0,908.0
1,001021707657411126,1239.0,Retraite,1239.0,1239.0
2,0010417079507GDPUB,1000.0,Allocation chômage / indemnités,1000.0,1000.0
5,001081708354301071,2000.0,Salaire ou revenu d'activité,2000.0,2000.0
6,0010917085404AL020,44030.0,Salaire ou revenu d'activité,44030.0,44030.0
...,...,...,...,...,...
1285438,9740317140124AL020,1560.0,Salaire ou revenu d'activité,1560.0,1560.0
1285442,9740317140152GDPUB,952.0,Retraite,952.0,952.0
1285443,974051714578111487,367.0,Autres (hors APL ou AL),168.0,168.0


### Calcul des prestations familiales et sociales mensuelles

In [384]:
values1=["Allocation d'adulte handicapé(AAH)", "Allocation d'éducation d'enfant handicapé (AEEH)", "Allocation de minimum vieillesse", "Allocation familiales", "Allocation Jeune Enfant (PAJE)", "Bourse étudiant", "Prime d'activité", "Revenu de solidarité active (RSA)", "Allocation journalière de présence parentale (AJPP)"]
filtered_df1 = dfd[dfd.Type.isin(values1)]
filtered_df1

,Numéro Unique,Ressources mensuelles\n,Type,Montant par type de ressource\n
1,001011403466511146,2134.0,Allocation familiales,424.0
4,0010117074877GDPUB,900.0,Allocation familiales,700.0
6,001021504965918214,944.0,Allocation familiales,129.0
7,001021504965918214,944.0,Revenu de solidarité active (RSA),815.0
14,0010417079394GDPUB,0.0,Allocation d'adulte handicapé(AAH),0.0
...,...,...,...,...
152663,9740617145930GDPUB,1943.0,Allocation d'éducation d'enfant handicapé (AEEH),718.0
152664,9740617145930GDPUB,1943.0,Allocation familiales,131.0
152666,9740617145930GDPUB,1943.0,Revenu de solidarité active (RSA),702.0
152669,974091715322011487,2121.0,Allocation familiales,38.0


In [385]:
shrank_values1 = filtered_df1.groupby(by='Numéro Unique').sum()['Montant par type de ressource\n']

merged_df1 = filtered_df1.merge(shrank_values1, left_on='Numéro Unique', right_index=True)

merged_df1 = merged_df1.rename(columns={'Montant par type de ressource\n_y': 'Prestations familiales et sociales mensuelles moyennes (PFS)'})

In [386]:
merged_df1

,Numéro Unique,Ressources mensuelles\n,Type,Montant par type de ressource\n_x,Prestations familiales et sociales mensuelles moyennes (PFS)
1,001011403466511146,2134.0,Allocation familiales,424.0,424.0
4,0010117074877GDPUB,900.0,Allocation familiales,700.0,700.0
6,001021504965918214,944.0,Allocation familiales,129.0,944.0
7,001021504965918214,944.0,Revenu de solidarité active (RSA),815.0,944.0
14,0010417079394GDPUB,0.0,Allocation d'adulte handicapé(AAH),0.0,0.0
...,...,...,...,...,...
152663,9740617145930GDPUB,1943.0,Allocation d'éducation d'enfant handicapé (AEEH),718.0,1551.0
152664,9740617145930GDPUB,1943.0,Allocation familiales,131.0,1551.0
152666,9740617145930GDPUB,1943.0,Revenu de solidarité active (RSA),702.0,1551.0
152669,974091715322011487,2121.0,Allocation familiales,38.0,38.0


In [387]:
merged_dfbis1=merged_df1.drop_duplicates(subset ="Numéro Unique")
merged_dfbis1

,Numéro Unique,Ressources mensuelles\n,Type,Montant par type de ressource\n_x,Prestations familiales et sociales mensuelles moyennes (PFS)
1,001011403466511146,2134.0,Allocation familiales,424.0,424.0
4,0010117074877GDPUB,900.0,Allocation familiales,700.0,700.0
6,001021504965918214,944.0,Allocation familiales,129.0,944.0
14,0010417079394GDPUB,0.0,Allocation d'adulte handicapé(AAH),0.0,0.0
29,001051708062718214,1102.0,Revenu de solidarité active (RSA),759.0,759.0
...,...,...,...,...,...
152655,973051301257111526,1611.0,Allocation familiales,296.0,1221.0
152659,9740217138112GDPUB,986.0,Allocation d'adulte handicapé(AAH),240.0,240.0
152663,9740617145930GDPUB,1943.0,Allocation d'éducation d'enfant handicapé (AEEH),718.0,1551.0
152669,974091715322011487,2121.0,Allocation familiales,38.0,38.0


In [402]:
bddbis = bdd.merge(merged_dfbis1, on='Numéro Unique', how='outer')
bddbis=bddbis.rename(columns={'Ressources mensuelles\n_x': 'RM Moyennes'})
bddbis=bddbis[['Numéro Unique', 'RM Moyennes','Ressources imposables mensuelles moyennes (RI)', 'Prestations familiales et sociales mensuelles moyennes (PFS)']]
bddbis

,Numéro Unique,RM Moyennes,Ressources imposables mensuelles moyennes (RI),Prestations familiales et sociales mensuelles moyennes (PFS)
0,001011403466511146,2134.0,1710.0,424.0
1,0010117074877GDPUB,900.0,200.0,700.0
2,001021504965918214,944.0,NaN,944.0
3,0010217076033GDPUB,1200.0,1200.0,NaN
4,001021707620301173,1200.0,1200.0,NaN
...,...,...,...,...
80874,9741117155529GDPUB,674.0,NaN,674.0
80875,974111715873511487,1725.0,1725.0,NaN
80876,9741216133916GDPUB,2000.0,2000.0,NaN
80877,9741216133920GDPUB,846.0,846.0,NaN


### Somme Revenus imposables et prestations

In [403]:
bddbis['Ressources imposables mensuelles moyennes (RI)'].fillna(0, inplace=True)
bddbis['Prestations familiales et sociales mensuelles moyennes (PFS)'].fillna(0, inplace=True)

In [404]:
bddbis

,Numéro Unique,RM Moyennes,Ressources imposables mensuelles moyennes (RI),Prestations familiales et sociales mensuelles moyennes (PFS)
0,001011403466511146,2134.0,1710.0,424.0
1,0010117074877GDPUB,900.0,200.0,700.0
2,001021504965918214,944.0,0.0,944.0
3,0010217076033GDPUB,1200.0,1200.0,0.0
4,001021707620301173,1200.0,1200.0,0.0
...,...,...,...,...
80874,9741117155529GDPUB,674.0,0.0,674.0
80875,974111715873511487,1725.0,1725.0,0.0
80876,9741216133916GDPUB,2000.0,2000.0,0.0
80877,9741216133920GDPUB,846.0,846.0,0.0


In [405]:
bddbis['Ri + PFS']=bddbis['Ressources imposables mensuelles moyennes (RI)']+bddbis['Prestations familiales et sociales mensuelles moyennes (PFS)']
bddbis

,Numéro Unique,RM Moyennes,Ressources imposables mensuelles moyennes (RI),Prestations familiales et sociales mensuelles moyennes (PFS),Ri + PFS
0,001011403466511146,2134.0,1710.0,424.0,2134.0
1,0010117074877GDPUB,900.0,200.0,700.0,900.0
2,001021504965918214,944.0,0.0,944.0,944.0
3,0010217076033GDPUB,1200.0,1200.0,0.0,1200.0
4,001021707620301173,1200.0,1200.0,0.0,1200.0
...,...,...,...,...,...
80874,9741117155529GDPUB,674.0,0.0,674.0,674.0
80875,974111715873511487,1725.0,1725.0,0.0,1725.0
80876,9741216133916GDPUB,2000.0,2000.0,0.0,2000.0
80877,9741216133920GDPUB,846.0,846.0,0.0,846.0


### Mise en place des tranches de ressources mensuelles

In [406]:
bddbis['Tranches de ressources mensuelles (RM)']=0

def decoupe(rl):
    for i in range(0, 5000, 50):
        if i<=rl and rl<i+50:
            return i
        elif rl>4950:
            return 4950

bddbis['Tranches de ressources mensuelles (RM)'] = bddbis['RM Moyennes'].apply(decoupe)
bddbis

,Numéro Unique,RM Moyennes,Ressources imposables mensuelles moyennes (RI),Prestations familiales et sociales mensuelles moyennes (PFS),Ri + PFS,Tranches de ressources mensuelles (RM)
0,001011403466511146,2134.0,1710.0,424.0,2134.0,2100.0
1,0010117074877GDPUB,900.0,200.0,700.0,900.0,900.0
2,001021504965918214,944.0,0.0,944.0,944.0,900.0
3,0010217076033GDPUB,1200.0,1200.0,0.0,1200.0,1200.0
4,001021707620301173,1200.0,1200.0,0.0,1200.0,1200.0
...,...,...,...,...,...,...
80874,9741117155529GDPUB,674.0,0.0,674.0,674.0,650.0
80875,974111715873511487,1725.0,1725.0,0.0,1725.0,1700.0
80876,9741216133916GDPUB,2000.0,2000.0,0.0,2000.0,2000.0
80877,9741216133920GDPUB,846.0,846.0,0.0,846.0,800.0


In [393]:
#bddbis['Tranches de ressources mensuelles'].dropna()

In [407]:
#bddbis.groupby(['Tranches de ressources mensuelles'])['Sales'].mean()
x=bddbis.groupby('Tranches de ressources mensuelles (RM)').mean().reset_index()
x

,Tranches de ressources mensuelles (RM),RM Moyennes,Ressources imposables mensuelles moyennes (RI),Prestations familiales et sociales mensuelles moyennes (PFS),Ri + PFS
0,0.0,1.844898,8.728571,1.648980,10.377551
1,50.0,78.186441,44.728814,42.796610,87.525424
2,100.0,118.508333,60.208333,86.941667,147.150000
3,150.0,176.809322,47.131356,134.745763,181.877119
4,200.0,218.787234,149.489362,84.581560,234.070922
...,...,...,...,...,...
94,4750.0,4771.400000,1611.600000,3159.800000,4771.400000
95,4800.0,4800.000000,4800.000000,0.000000,4800.000000
96,4850.0,4861.000000,4861.000000,0.000000,4861.000000
97,4900.0,4936.000000,4014.000000,922.000000,4936.000000


In [395]:
#x['Nombre de demandeurs'] = bddbis.groupby('Tranches de ressources mensuelles').count()
y=bddbis['Tranches de ressources mensuelles (RM)'].value_counts()
y=y.reset_index()
y

,index,Tranches de ressources mensuelles (RM)
0,800.0,5280
1,1200.0,3785
2,450.0,3773
3,1300.0,3352
4,900.0,3186
...,...,...
94,4800.0,2
95,4900.0,2
96,4450.0,2
97,4600.0,2


In [396]:
y=y.rename(columns={'index': 'Tranches de ressources mensuelles (RM)', "Tranches de ressources mensuelles (RM)": "Nombre de demandeurs"})

In [397]:
y

,Tranches de ressources mensuelles (RM),Nombre de demandeurs
0,800.0,5280
1,1200.0,3785
2,450.0,3773
3,1300.0,3352
4,900.0,3186
...,...,...
94,4800.0,2
95,4900.0,2
96,4450.0,2
97,4600.0,2


In [398]:
final = x.merge(y, on='Tranches de ressources mensuelles (RM)', how='left')
final

,Tranches de ressources mensuelles (RM),RM Moyennes,Ressources imposables mensuelles moyennes (RI),Prestations familiales et sociales mensuelles moyennes (PFS),Ri + PFS,Nombre de demandeurs
0,0.0,1.844898,8.728571,1.648980,10.377551,490
1,50.0,78.186441,44.728814,42.796610,87.525424,59
2,100.0,118.508333,60.208333,86.941667,147.150000,240
3,150.0,176.809322,47.131356,134.745763,181.877119,236
4,200.0,218.787234,149.489362,84.581560,234.070922,141
...,...,...,...,...,...,...
94,4750.0,4771.400000,1611.600000,3159.800000,4771.400000,5
95,4800.0,4800.000000,4800.000000,0.000000,4800.000000,2
96,4850.0,4861.000000,4861.000000,0.000000,4861.000000,1
97,4900.0,4936.000000,4014.000000,922.000000,4936.000000,2


In [399]:
final=final[['Tranches de ressources mensuelles (RM)', 'Nombre de demandeurs', 'RM Moyennes', 'Prestations familiales et sociales mensuelles moyennes (PFS)', 'Ressources imposables mensuelles moyennes (RI)', 'Ri + PFS']]

In [400]:
final=final.reset_index(drop=True)
final

,Tranches de ressources mensuelles (RM),Nombre de demandeurs,RM Moyennes,Prestations familiales et sociales mensuelles moyennes (PFS),Ressources imposables mensuelles moyennes (RI),Ri + PFS
0,0.0,490,1.844898,1.648980,8.728571,10.377551
1,50.0,59,78.186441,42.796610,44.728814,87.525424
2,100.0,240,118.508333,86.941667,60.208333,147.150000
3,150.0,236,176.809322,134.745763,47.131356,181.877119
4,200.0,141,218.787234,84.581560,149.489362,234.070922
...,...,...,...,...,...,...
94,4750.0,5,4771.400000,3159.800000,1611.600000,4771.400000
95,4800.0,2,4800.000000,0.000000,4800.000000,4800.000000
96,4850.0,1,4861.000000,0.000000,4861.000000,4861.000000
97,4900.0,2,4936.000000,922.000000,4014.000000,4936.000000


In [401]:
file_name = 'Analyse des ressources v4.xlsx'
final.to_excel(file_name)

### Tableaux croisés RI / PFS

In [431]:
bddbis['Tranches de RI']=0
bddbis['Tranches de PFS']=0

def decoupe2(rl):
    for i in range(0, 2050, 50):
        if i<=rl and rl<i+50:
            return i
        elif rl>2001:
            return np.NaN

bddbis['Tranches de RI'] = bddbis['Ressources imposables mensuelles moyennes (RI)'].apply(decoupe2)
bddbis['Tranches de PFS'] = bddbis['Prestations familiales et sociales mensuelles moyennes (PFS)'].apply(decoupe2)
bddbis

,Numéro Unique,RM Moyennes,Ressources imposables mensuelles moyennes (RI),Prestations familiales et sociales mensuelles moyennes (PFS),Ri + PFS,Tranches de ressources mensuelles (RM),Tranches de RI,Tranches de PFS
0,001011403466511146,2134.0,1710.0,424.0,2134.0,2100.0,1700.0,400.0
1,0010117074877GDPUB,900.0,200.0,700.0,900.0,900.0,200.0,700.0
2,001021504965918214,944.0,0.0,944.0,944.0,900.0,0.0,900.0
3,0010217076033GDPUB,1200.0,1200.0,0.0,1200.0,1200.0,1200.0,0.0
4,001021707620301173,1200.0,1200.0,0.0,1200.0,1200.0,1200.0,0.0
...,...,...,...,...,...,...,...,...
80874,9741117155529GDPUB,674.0,0.0,674.0,674.0,650.0,0.0,650.0
80875,974111715873511487,1725.0,1725.0,0.0,1725.0,1700.0,1700.0,0.0
80876,9741216133916GDPUB,2000.0,2000.0,0.0,2000.0,2000.0,2000.0,0.0
80877,9741216133920GDPUB,846.0,846.0,0.0,846.0,800.0,800.0,0.0


In [447]:
tableau=bddbis[['Tranches de RI', 'Tranches de PFS']]
tableau=tableau.dropna()
tableau

,Tranches de RI,Tranches de PFS
0,1700.0,400.0
1,200.0,700.0
2,0.0,900.0
3,1200.0,0.0
4,1200.0,0.0
...,...,...
80874,0.0,650.0
80875,1700.0,0.0
80876,2000.0,0.0
80877,800.0,0.0


In [449]:
shrank = tableau.groupby(by='Tranches de RI').count()['Tranches de PFS']
shrank

Tranches de RI
0.0       13589
50.0        347
100.0      1227
150.0       737
200.0       837
250.0       583
300.0       680
350.0       662
400.0       650
450.0      1060
500.0      1558
550.0       761
600.0      1223
650.0      1263
700.0      1569
750.0      1589
800.0      2406
850.0      1831
900.0      2305
950.0      2068
1000.0     2420
1050.0     1776
1100.0     2790
1150.0     2508
1200.0     3849
1250.0     2374
1300.0     3084
1350.0     2165
1400.0     2844
1450.0     1990
1500.0     2952
1550.0     1471
1600.0     2083
1650.0     1212
1700.0     1500
1750.0      868
1800.0     1323
1850.0      651
1900.0      802
1950.0      496
2000.0      538
Name: Tranches de PFS, dtype: int64

In [441]:
def croix(c1, c2):
    for i in range(0, 76640):
        for j in range(0, 76640):
            if c1 == 0:
                if c2 ==0:
                    df[i][j] =+1

        return 0.3
    else:
        return 0

base['C1'] = base['Rapport RL'].apply(c1)
base

,index,Tranches de RI
0,0.0,13589
1,1200.0,3849
2,1300.0,3084
3,1500.0,2952
4,1400.0,2844
5,1100.0,2790
6,1150.0,2508
7,1000.0,2420
8,800.0,2406
9,1250.0,2374


In [439]:
z.reset_index()

,Tranches de RI,0
0,0.0,13589
1,50.0,347
2,100.0,1227
3,150.0,737
4,200.0,837
5,250.0,583
6,300.0,680
7,350.0,662
8,400.0,650
9,450.0,1060


In [437]:
z

Tranches de RI
0.0       13589
50.0        347
100.0      1227
150.0       737
200.0       837
250.0       583
300.0       680
350.0       662
400.0       650
450.0      1060
500.0      1558
550.0       761
600.0      1223
650.0      1263
700.0      1569
750.0      1589
800.0      2406
850.0      1831
900.0      2305
950.0      2068
1000.0     2420
1050.0     1776
1100.0     2790
1150.0     2508
1200.0     3849
1250.0     2374
1300.0     3084
1350.0     2165
1400.0     2844
1450.0     1990
1500.0     2952
1550.0     1471
1600.0     2083
1650.0     1212
1700.0     1500
1750.0      868
1800.0     1323
1850.0      651
1900.0      802
1950.0      496
2000.0      538
dtype: int64

In [ ]:
pd.melt(df, id_vars=['A'], value_vars=['B'])